In [1]:
import networkx as nx
import osmnx as ox
import requests
import sys,os,os.path
import pandas as pd
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

ox.config(use_cache=True, log_console=True)
ox.__version__

'0.16.1'

In [2]:
stations = pd.read_csv('../../data/stations.csv')

In [3]:
stations.head()

,station_cd,station_g_cd,station_name,station_name_k,station_name_r,line_cd,pref_cd,post,address,lon,lat,open_ymd,close_ymd,e_status,e_sort
0,1110101,1110101,函館,NaN,NaN,11101,1,040-0063,北海道函館市若松町１２-１３,140.726413,41.773709,1902-12-10,0000-00-00,0,1110101
1,1110102,1110102,五稜郭,NaN,NaN,11101,1,041-0813,函館市亀田本町,140.733539,41.803557,0000-00-00,0000-00-00,0,1110102
2,1110103,1110103,桔梗,NaN,NaN,11101,1,041-0801,北海道函館市桔梗３丁目４１-３６,140.722952,41.846457,1902-12-10,0000-00-00,0,1110103
3,1110104,1110104,大中山,NaN,NaN,11101,1,041-1121,亀田郡七飯町大字大中山,140.713580,41.864641,0000-00-00,0000-00-00,0,1110104
4,1110105,1110105,七飯,NaN,NaN,11101,1,041-1111,亀田郡七飯町字本町,140.688556,41.886971,0000-00-00,0000-00-00,0,1110105


In [4]:
# 日本語・ローマ字変換ライブラリ `pykakasi` をインストール
!pip install git+https://github.com/miurahr/pykakasi

  Cloning https://github.com/miurahr/pykakasi to /tmp/pip-req-build-jwszaii4
  ERROR: Error [Errno 2] No such file or directory: 'git' while executing command git clone -q https://github.com/miurahr/pykakasi /tmp/pip-req-build-jwszaii4
ERROR: Cannot find command 'git' - do you have 'git' installed and in your PATH?


In [5]:
from pykakasi import kakasi

kakasi = kakasi()

kakasi.setMode('H', 'a')
kakasi.setMode('K', 'a')
kakasi.setMode('J', 'a')

conv = kakasi.getConverter()

ModuleNotFoundError: No module named 'pykakasi'

In [ ]:
# 駅名のローマ字名を付与
stations['station_name_roman'] = stations['station_name'].apply(lambda name: conv.do(name))
stations

In [8]:
target_stations = stations[(stations['line_cd'] == 11312) | (stations['line_cd'] == 24001)]

In [9]:
len(target_stations[target_stations['line_cd'] == 11312]['station_name'])

24

In [10]:
import folium

In [13]:
center_lat, center_lon = (35.651820534474,139.54461236426)

_map = folium.Map(location=[center_lat, center_lon], zoom_start=12)

for _, station in target_stations.iterrows():
    color = 'red' if station['line_cd'] == 11312 else 'pink'
    folium.Marker([station.lat, station.lon], popup=str(station['station_name']), icon=folium.Icon(color=color)).add_to(_map)

_map

In [ ]:
#駅の800m範囲内のグラフ構造の配列を取得
def get_stats(data):
    wurster_hall = (data['lat'], data['lon'])
    one_mile = 800 #meters
    G = ox.graph_from_point(wurster_hall, dist=one_mile, network_type='drive')

    # what sized area does our network cover in square meters?
    G_proj = ox.project_graph(G)
    nodes_proj = ox.graph_to_gdfs(G_proj, edges=False)
    graph_area_m = nodes_proj.unary_union.convex_hull.area
    
    # show some basic stats about the network
    stats = ox.basic_stats(G_proj, area=graph_area_m, clean_intersects=True, circuity_dist='euclidean')
    return pd.Series(stats)
    

In [ ]:
#駅の800m範囲内のグラフ構造の配列を取得
stats_list = target_stations.apply(get_stats, axis=1)
stats_list

In [ ]:
stats_list[0]

In [ ]:
target_stations_with_stats = pd.concat([target_stations, stats_list], axis=1)

In [ ]:
target_stations_with_stats.to_csv('../../data/target_stations_with_stats')

In [ ]:
target_stations_with_stats = pd.read_csv('../../data/target_stations_with_stats.csv')

In [ ]:
# 路線ごとの交差点数、道路数のグラフ化
ax = None
colors = {11312: 'red', 24001: 'pink'}
for i, station in target_stations_with_stats.groupby('line_cd'):
    station.plot.scatter(x='n', y='m', color=colors[i], label='cluster{i}', ax=ax)

In [ ]:
plt.hist([chuo_stations['n'], keio_stations['n']], bins = 20, color=['red', 'pink'])


In [ ]:
plt.hist([chuo_stations['m'], keio_stations['m']], bins = 20, color=['red', 'pink'])


In [ ]:
# k-means法を使うためのインポート
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# データの列の絞り込み
stations_sub = target_stations_with_stats[['n','m']]

# 標準化
sc = StandardScaler()
sc.fit(stations_sub)
stations_sub_std = sc.transform(stations_sub)

In [ ]:
# KMeansクラスの初期化
kmeans = KMeans(init='random', n_clusters=2, random_state=0)

# クラスターの重心を計算
kmeans.fit(stations_sub_std)

# クラスター番号をpandasのSeriesオブジェクトに変換
labels = pd.Series(kmeans.labels_, name='cluster_number', index=target_stations_with_stats.index)

# クラスター番号と件数を表示
print(labels.value_counts(sort=False))

# グラフを描画
ax = labels.value_counts(sort=False).plot(kind='bar')
ax.set_xlabel('cluster number')
ax.set_ylabel('count')

In [ ]:
# エルボー方による推定。クラスター数を1から20に増やして、それぞれの距離の総和を求める
dist_list =[]
for i in range(1,20):
    kmeans= KMeans(n_clusters=i, init='random', random_state=0)
    kmeans.fit(stations_sub_std)
    dist_list.append(kmeans.inertia_)
    
# グラフを表示
plt.plot(range(1,20), dist_list,marker='+')
plt.xlabel('Number of clusters')
plt.ylabel('Distortion')

In [ ]:
# 駅のデータにクラスター番号のデータを結合
stations_with_cluster = pd.concat([target_stations_with_stats, pd.Series(labels, index=target_stations_with_stats.index)], axis=1)

# 先頭の5行を表示
stations_with_cluster.head()

In [ ]:
stations_0 = stations_with_cluster[stations_with_cluster['cluster_number'] == 0]
stations_1 = stations_with_cluster[stations_with_cluster['cluster_number'] == 1]
# クラスタリング結果のグラフ化
ax = None
colors = ['red', 'pink']
for i, station in stations_with_cluster.groupby('cluster_number'):
    station.plot.scatter(x='n', y='m', color=colors[i], label='cluster{i}', ax=ax)

In [ ]:
# クラスタ番号と年齢層を軸に集計し、年齢層を列に設定
cross_cluster_line_cd = stations_with_cluster.groupby(['cluster_number', 'line_cd']).size().unstack().fillna(0)
cross_cluster_line_cd

In [ ]:
sns.heatmap(cross_cluster_line_cd.apply(lambda x : x/x.sum(), axis=1), cmap='Blues')

In [ ]:
print([station['station_name'] for _, station in stations_with_cluster.iterrows() if (station['cluster_number'] == 0)])

In [ ]:
print([station['station_name'] for _, station in stations_with_cluster.iterrows() if (station['cluster_number'] == 1)])